In [9]:
import numpy as np
np.random.seed(42)
import random
random.seed(42)
from sklearn import datasets
import matplotlib.pyplot as plt

In [10]:
categories = ['comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey']
train_dataset = datasets.fetch_20newsgroups(subset = 'train', categories = categories, shuffle = True, random_state = None)
test_dataset = datasets.fetch_20newsgroups(subset = 'test', categories = categories, shuffle = True, random_state = None)

In [11]:
from nltk import WordNetLemmatizer
from nltk import pos_tag
import nltk
from nltk.corpus import wordnet
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
#Define Stopwords, here we use stopwords in text package
stop_words=text.ENGLISH_STOP_WORDS
print("number of stop_words in sklearn text:%s" % len(stop_words))

number of stop_words in sklearn text:318


In [12]:
#implement new dataset
categories_computer=['comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware']
categories_rec=['rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey']
computer_train=datasets.fetch_20newsgroups(subset = 'train', categories = categories_computer, shuffle = True, random_state = None).data
computer_test=datasets.fetch_20newsgroups(subset = 'test', categories = categories_computer, shuffle = True, random_state = None).data
recreation_train=datasets.fetch_20newsgroups(subset = 'train', categories = categories_rec, shuffle = True, random_state = None).data
recreation_test=datasets.fetch_20newsgroups(subset = 'test', categories = categories_rec, shuffle = True, random_state = None).data

In [13]:
#implement new dataset cont'd: combine train and test together and save as X_overall
X_train=computer_train+recreation_train
X_test=computer_test+recreation_test
X_overall=X_train+X_test
Y_train=[1]*len(computer_train)+[0]*len(recreation_train)
Y_test=[1]*len(computer_test)+[0]*len(recreation_test)
Y_overall=Y_train+Y_test
print(np.shape(X_overall))

MemoryError: Unable to allocate 4.72 GiB for an array with shape (7882,) and data type <U160616

In [ ]:
#Lemmatization
lemmatizer=WordNetLemmatizer()
#Define lemmatizer pos_tag to deal with adj, verb, noun and adv separately
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

#Preprocessing: exclude some symbols and numbers in each sentence
exclude_sign="~#$%^&*(){}[]<>|+=1234567890"
replace_sign="@,.?!-;"
def preprocessing(data):
    processed_sentence=[]
    for i in range(len(data)):
        sentence=data[i]
        for c in exclude_sign:
            sentence=sentence.replace(c,"")
        for c in replace_sign:
            sentence=sentence.replace(c," ")
        processed_token=[lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)]
        processed_sentence.append(" ".join(processed_token))
    return processed_sentence



#operation of lemmaization

processed_data=preprocessing(X_overall)







In [8]:
#Define class lemmatization for pipeline step
class Lemmatization():
    def __init__ (self, lemmatize_shold):
        self.lemmatize_shold=lemmatize_shold
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    def fit(self, X):
        if self.lemmatize_shold==0:
            fitted= X
        elif self.lemmatize_shold==1:
            fitted= preprocessing(X)
        return fitted
    def transform(self,fitted):
        return fitted
#Define class train_test split for pipeline step

        
    

In [14]:
#lemmatize_class_test=Lemmatization(1)
#lemmatize_class_test.fit_transform(X_overall)

["From : kaufman Xenon Stanford EDU Marc T Kaufman Subject : Re : TIFF : philosophical significance of Reply To : kaufman CS Stanford EDU Organization : CS Department Stanford University California USA Lines : joachim kih no joachim lous writes : ulrich galki toppoint de write : According to the TIFF Specification the TIFF `` version number '' byte have be chosen for it `` deep philosophical significance '' Last week I read the Hitchhikers Guide To The Galaxy and rotfl the second time After million of year of calculation the second best computer of all time reveals that be the answer to the question about life the universe and everything Is this actually how they picked the number Yes Does anyone have any other suggestion where the come from At this moment the King who have be for some time busily write in his note book call out `` Silence `` and read out from his book '' Rule Forty two All person more than a mile high to leave the court `` Everybody look at Alice `` I 'm not a mile hi

In [7]:
print(type(processed_data))

processed_train_data=processed_data[0:len(X_train)]
print(np.shape(processed_train_data))

print(np.shape(Y_train))

<class 'list'>
(4732,)
(4732,)


In [8]:
#define classifier function SVM, LR, GaussianNB

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.model_selection import cross_val_score



In [9]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF



In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

#Define a pipeline object which mnust be a tuple

pipe=Pipeline([
               ('vectorize', CountVectorizer(min_df=3,stop_words='english')),
               ('TFIDF',TfidfTransformer()),
               ('dimensionality_reduction', NMF(n_components=50)),
               ('classification', LogisticRegression(penalty='l1',C=100,solver='liblinear'))
              ])
#     parameter_grid=[
#         {
#             'vectorize__min_df': [3,5]

#         }

#     ]



    

In [11]:
#With header and footer
pipe.fit(processed_train_data,Y_train)
cvs=np.average(cross_val_score(pipe,processed_train_data,Y_train, cv=5))
print(cvs)

0.9708383657304364


In [13]:
parameter_grid=[{
                 'vectorize__min_df': [3,5],
                 'dimensionality_reduction':[NMF(),TruncatedSVD()],
                 'classification': [svm.SVC(gamma=1),LogisticRegression(penalty='l1',C=100,solver='liblinear'),LogisticRegression(penalty='l2',C=100,solver='liblinear'),GaussianNB()]
    
               }]

In [14]:
grid = GridSearchCV(pipe, cv=5, n_jobs=-1, param_grid=parameter_grid, scoring='accuracy')






In [ ]:
grid.fit(processed_train_data, Y_train)

In [ ]:
print(grid.best_score_)
print('The best configuration is:')
print(grid.best_params_)